In [1]:
#!jupyter nbconvert --to script pythiaTesting.ipynb
import sys
import ROOT as RT
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, DelphesSchema


Welcome to JupyROOT 6.24/06


In [2]:
def getEvents(fileURL, treename):
    return NanoEventsFactory.from_root(fileURL, treepath = treename, schemaclass = DelphesSchema).events()

def histoSettings(histos):
    for key in histos:
        histos[key].Sumw2()
        histos[key].SetDirectory(0)


In [ ]:
PIDof = {'LLP': 1500001, 'mu': 13, 'b': 5, 'K': 313}

filedir   = "root://cms-xrd-global.cern.ch//store/user/christiw/BDecay_delphes/v1/"
filenames = {'1e4mm': "m1_ct10000mm.root",
             '1e3mm': "m1_ct1000mm.root",
             '1e2mm': "m1_ct100mm.root",
             '1e1mm': "m1_ct10mm.root"}
treename  = "Delphes"

datasets = {'1e4mm': getEvents(filedir+filenames['1e4mm'], treename)}

#datasets = {}
#for key in filenames:
#    datasets[key] = getEvents(filedir+filenames[key], treename)


In [ ]:
histos = {}
for key in PIDof:
    histos[key+'PT']  = RT.TH1F(key+'P_{T}', 'Events',30,0,100)
    histos[key+'Eta'] = RT.TH1F(key+'\eta', 'Events',30,0,5)
    histos[key+'Phi'] = RT.TH1F(key+'\phi', 'Events',30,-5,5)
    
#histoSettings(histos)

In [ ]:
'''
def muonReq(event, nMuons, muonsP4):
    
    muonList = event[abs(event.PID) == PIDof['mu']]
    for ij in range(len(muonList)):
        #get the kinematics and select the muon   
        mp4 = RT.TLorentzVector()
        mp4.SetPtEtaPhiM(muonList.PT[ij],
                         muonList.Eta[ij],
                         muonList.Phi[ij],
                         0)
        
        if mp4.Pt()<8 : continue
        #count selected muons
        nMuons +=1
        muonsP4.append(mp4)
        
    if nMuons < 1 : raise Exception('not enough muons!')
    return nMuons
'''

def muonReq(event, muonsP4):
    
    muonList = event[abs(event.PID) == PIDof['mu']]
    #muonList = muonList[muonList.PT > 8]
    muonList = muonList[muonList.Eta < 2.4]
    
    nMuons = len(muonList)
    if nMuons < 1 : raise Exception('not enough muons!')
        
    for ij in range(nMuons):
        #get the kinematics and select the muon   
        mp4 = RT.TLorentzVector()
        mp4.SetPtEtaPhiM(muonList.PT[ij],
                         muonList.Eta[ij],
                         muonList.Phi[ij],
                         0)
        
        muonsP4.append(mp4)
        

In [ ]:
'''def BmesonReq(event, nBmesons, BmesonsP4):
    
    BmesonList = event[abs(event.PID) == PIDof['b']]
    for ij in range(len(BmesonList)):
        #get the kinematics and select the Bmeson (~sorta)
        bp4 = RT.TLorentzVector()
        bp4.SetPtEtaPhiM(BmesonList.PT[ij],
                         BmesonList.Eta[ij],
                         BmesonList.Phi[ij],
                         0)
        
        #count selected bs
        nBmesons +=1
        BmesonsP4.append(bp4)
    if nBmesons < 2 : raise Exception('not enough bs!')
    return nBmesons'''

def BmesonReq(event, BmesonsP4):
    
    BmesonList = event[abs(event.PID) == PIDof['b']]
    
    nBmesons = len(BmesonList)
    if nBmesons < 2 : raise Exception('not enough bs!')
    
    for ij in range(nBmesons):
        #get the kinematics and select the Bmeson (~sorta)
        bp4 = RT.TLorentzVector()
        bp4.SetPtEtaPhiM(BmesonList.PT[ij],
                         BmesonList.Eta[ij],
                         BmesonList.Phi[ij],
                         0)
    
        BmesonsP4.append(bp4)
        


In [ ]:
def kaonReq(event, KaonsP4):
    
    KaonList = event[abs(event.PID) == PIDof['K']]

    nKaons = len(KaonList)
    if nKaons < 1 : raise Exception('not enough Kaons!')
    
    for ij in range(nKaons):
        #get the kinematics and select the Kaon (~sorta)
        kp4 = RT.TLorentzVector()
        kp4.SetPtEtaPhiM(KaonList.PT[ij],
                         KaonList.Eta[ij],
                         KaonList.Phi[ij],
                         0)  

        KaonsP4.append(kp4)



In [ ]:
def LLPReq(event, llpP4):
    
    llpList = event[abs(event.PID) == PIDof['LLP']]

    nLLPs = len(llpList)
    
    for ij in range(nLLPs):
        #get the kinematics and select the Kaon (~sorta)
        lp4 = RT.TLorentzVector()
        lp4.SetPtEtaPhiM(llpList.PT[ij],
                         llpList.Eta[ij],
                         llpList.Phi[ij],
                         0)  

        llpP4.append(lp4)
        
    return nLLPs


In [ ]:
def inMuonChambers(event, llpP4):
    
    llpList = event[abs(event.PID) == PIDof['LLP']]
    llpList = llpList[]
    
    
    nLLPs = len(llpList)
    
    for ij in range(nLLPs):
        #get the kinematics and select the Kaon (~sorta)
        lp4 = RT.TLorentzVector()
        lp4.SetPtEtaPhiM(llpList.PT[ij],
                         llpList.Eta[ij],
                         llpList.Phi[ij],
                         0)  

        llpP4.append(lp4)
        
    return nLLPs

In [35]:
ctau = '1e4mm'
totalEntries = len(datasets[ctau])
nLLPsGuess = 0
nLLPsReal = 0

for i in range(totalEntries):
    event = datasets[ctau].Particle[i]
    if i%100==0 : sys.stdout.write('\r [ %d/100 ] done' %(int(float(100.*i)/float(totalEntries))) )
    
    llpP4 = []
    nLLPsReal += LLPReq(event, llpP4)
    
    for ij in range(0,len(llpP4)):
        histos['LLPPT'].Fill(llpP4[ij].Pt())
        histos['LLPEta'].Fill(llpP4[ij].Eta())
        histos['LLPPhi'].Fill(llpP4[ij].Phi())
    
    muonsP4, BmesonsP4, KaonsP4 = [], [], []
    
    #require at least 1 muon
    try: muonReq(event, muonsP4)
    except: 
        #traceback.print_exc()
        continue   
    
    #require at least 2 Bmesons
    try: BmesonReq(event, BmesonsP4)
    except: 
        #traceback.print_exc()
        continue  
    
    #require at leat 1 Kaon
    try: kaonReq(event, KaonsP4)
    except: 
        #traceback.print_exc()
        continue   
    
    nLLPsGuess +=1
    for ij in range(0,len(muonsP4)):
        histos['muPT'].Fill(muonsP4[ij].Pt())
        histos['muEta'].Fill(muonsP4[ij].Eta())
        histos['muPhi'].Fill(muonsP4[ij].Phi())

    for ij in range(0,len(BmesonsP4)):
        histos['bPT'].Fill(BmesonsP4[ij].Pt())
        histos['bEta'].Fill(BmesonsP4[ij].Eta())
        histos['bPhi'].Fill(BmesonsP4[ij].Phi())

    for ij in range(0,len(KaonsP4)):
        histos['KPT'].Fill(KaonsP4[ij].Pt())
        histos['KEta'].Fill(KaonsP4[ij].Eta())
        histos['KPhi'].Fill(KaonsP4[ij].Phi())
        

  
   

 [ 99/100 ] done

In [28]:
outFileName = 'kinematics.pdf'
canvas = RT.TCanvas("canvas")
canvas.SetLogy(True)
canvas.Print(outFileName + "[")
for key in histos:
    histos[key].Draw()
    canvas.Print(outFileName)
    canvas.Clear()
    
canvas.Print(outFileName + "]") #closes pdf
canvas.Close() #closes canvas

Info in <TCanvas::Print>: pdf file kinematics.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file kinematics.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file kinematics.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file kinematics.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file kinematics.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file kinematics.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file kinematics.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file kinematics.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file kinematics.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file kinematics.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file kinematics.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file kinematics.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file kinematics.pdf
Info in <TCanvas::Print>: pdf file kinematics

In [33]:
nLLPsGuess/nLLPsReal

0.1204966771598461

In [36]:
nLLPsGuess/nLLPsReal

0.20627841902763203

In [58]:
from particle import Particle


Particle.findall(pdg_name='K')


[<Particle: name="K0", pdgid=311, mass=497.611 ± 0.013 MeV>,
 <Particle: name="K~0", pdgid=-311, mass=497.611 ± 0.013 MeV>,
 <Particle: name="K+", pdgid=321, mass=493.677 ± 0.016 MeV>,
 <Particle: name="K-", pdgid=-321, mass=493.677 ± 0.016 MeV>]